# **[필수과제1] 유사도**
- 다른 리뷰로 CountVectorize, TF-IDF 진행
- 유사도 어떻게 나오는지 확인
- 어떤 차이 발생하는지 확인

---



In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('movie_rv.csv')

In [3]:
df['document'] = df['document'].fillna(0)
df['document'] = df['document'].astype(str)

In [4]:
df_docu = df['document'].iloc[:50000]

In [5]:
df_docu

0                                      아 더빙.. 진짜 짜증나네요 목소리
1                        흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                        너무재밓었다그래서보는것을추천한다
3                            교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4        사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                               ...                        
49995    "이게 소위 스페인식 Neo-realism 이란 건가? - 물론, ""아동 판타지"...
49996                                     보지마셈 저 믿으세요 보지마셈
49997                                                 최고졸작
49998                                    재밌고 좋았음 10자 쓰라고?!
49999                                 소박하지만 잔잔한 감동을 주는 영화.
Name: document, Length: 50000, dtype: object

In [6]:
df_docu.isna().sum()

0

## CountVectorize

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
re_cv = CountVectorizer(max_features=1000)

In [10]:
re_dtm = re_cv.fit_transform(df_docu)

In [11]:
re_dtm

<50000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 124124 stored elements in Compressed Sparse Row format>

In [12]:
re_cv.get_feature_names_out()[:100]

array(['0점', '0점은', '10', '10자', '10점', '1점', '1점도', '2점', '3류', '3점',
       '4점', '80년대', '90년대', '9점', 'b급', 'good', 'oo', 'ooo', 'ooo기',
       'ost', 'the', 'tv', 'ㄷㄷ', 'ㅅㅂ', 'ㅇㅇ', 'ㅉㅉ', 'ㅋㅋ', 'ㅋㅋㅋ', 'ㅋㅋㅋㅋ',
       'ㅋㅋㅋㅋㅋ', 'ㅎㅎ', 'ㅎㅎㅎ', 'ㅜㅜ', 'ㅠㅠ', 'ㅠㅠㅠ', 'ㅡㅡ', '가끔', '가는', '가볍게',
       '가서', '가슴', '가슴이', '가장', '가지고', '가진', '각본', '간만에', '갈수록', '감독',
       '감독님', '감독은', '감독의', '감독이', '감동', '감동과', '감동도', '감동을', '감동이',
       '감동적이고', '감동적인', '감사합니다', '갑자기', '강추', '같네요', '같다', '같습니다', '같아서',
       '같아요', '같은', '같은데', '같음', '같이', '개봉', '개뿔', '개연성', '개인적으로', '거의',
       '건지', '걸작', '겁나', '것도', '것은', '것을', '것이', '것이다', '게다가', '결국', '결말',
       '결말도', '결말은', '결말이', '계속', '공감이', '공포', '공포영화', '과연', '괜찮은', '괜히',
       '굉장히', '구성'], dtype=object)

In [14]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 35.2 MB/s eta 0:00:00


In [15]:
from konlpy.tag import Okt

In [16]:
tw_tag = Okt()

In [17]:
def my_tokenizer(doc):
    return[
        token
        for token, pos in tw_tag.pos(doc)
        if pos in ['Noun','Verb']
    ]

In [18]:
print('결과', my_tokenizer(df.document[5]))

결과 ['막', '걸음', '마', '뗀', '세', '초등학교', '학년', '생인', '살용', '영화', '반개', '움']


In [19]:
re_cv = CountVectorizer(max_features=1000, tokenizer=my_tokenizer)

In [20]:
re_dtm = re_cv.fit_transform(df_docu)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [21]:
for word, count in zip(re_cv.get_feature_names_out(), re_dtm[5].toarray()[0]):
    if count > 0:
        print(word, count, end=',')

마 1,막 1,반개 1,세 1,영화 1,움 1,

## TF-IDF

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer

In [23]:
transformer = TfidfTransformer()
transformer

TfidfTransformer()

In [24]:
rv_tfidf = transformer.fit_transform(re_dtm)

In [25]:
rv_tfidf

<50000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 227438 stored elements in Compressed Sparse Row format>

In [26]:
tf = TfidfTransformer()
rv_tfidf = tf.fit_transform(re_dtm)

## Cosine_Similarity 유사도 확인

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
st = len(df_docu[5])//2

In [29]:
source = df_docu[5][-st:]

In [30]:
df_docu[5]

'막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.'

In [31]:
st

22

In [32]:
source

' 8살용영화.ㅋㅋㅋ...별반개도 아까움.'

In [33]:
source_cv = re_cv.transform([source])

In [34]:
source_cv

<1x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [35]:
sim_res_cv = cosine_similarity(source_cv,re_dtm)

In [36]:
sim_res_cv

array([[0.        , 0.21821789, 0.        , ..., 0.        , 0.        ,
        0.33333333]])

In [40]:
source_idf = tf.transform(source_cv)

In [41]:
source_idf

<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [42]:
sim_res_tfidf = cosine_similarity(source_idf, rv_tfidf)

In [43]:
sim_res_tfidf

array([[0.        , 0.03532273, 0.        , ..., 0.        , 0.        ,
        0.06436142]])

In [46]:
print('[CountVectorize] 가장 유사한 리뷰는?',np.argmax(sim_res_cv[0]))

[CountVectorize] 가장 유사한 리뷰는? 6728


In [45]:
print('[TF-IDF] 가장 유사한 리뷰는?',np.argmax(sim_res_tfidf[0]))

[TF-IDF] 가장 유사한 리뷰는? 11824


In [47]:
df_docu[6728]

'영화 무서운게 아니고 징그러움.'

In [48]:
df_docu[11824]

'반개도 아까움..67ㅕ465437'